In [1]:
from modelscope import BertTokenizer, BertForMaskedLM
tokenizer = BertTokenizer.from_pretrained('AI-ModelScope/bert-base-uncased')
model = BertForMaskedLM.from_pretrained('AI-ModelScope/bert-base-uncased')
model.eval()

/data/miniconda/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-05 13:19:39,227 - modelscope - WARNING - Using branch: master as version is unstable, use with caution


2025-05-05 13:19:39,509 - modelscope - INFO - Got 11 files, start to download ...
Processing 11 items:   0%|          | 0.00/11.0 [00:00<?, ?it/s]



























Processing 11 items:   9%|▉         | 1.00/11.0 [00:00<00:03, 3.24it/s]












Processing 11 items:  27%|██▋       | 3.00/11.0 [00:00<00:00, 8.49it/s]


































Processing 11 items:  82%|████████▏ | 9.00/11.0 [00:00<00:00, 12.0it/s]






















































































































































































































































































































































































































































































































































































2025-05-05 13:21:05,091 - modelscope - INFO - Got 5 files, start to download ...
Processing 5 items:   0%|          | 0.00/5.00 [00:00<?, ?it/s]

























































































































































































































































































































































































































































































































































































































































































































































































































































































BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [2]:
# check
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
print(output)

MaskedLMOutput(loss=None, logits=tensor([[[ -9.3893,  -9.3195,  -9.1902,  ...,  -8.1698,  -8.5670,  -5.9066],
         [-10.8334, -10.7239, -10.7502,  ...,  -9.9730,  -9.6572,  -8.2451],
         [-11.6256, -11.6355, -11.4426,  ..., -10.0851, -11.0916,  -8.9086],
         ...,
         [ -8.9649,  -8.7341,  -9.0515,  ...,  -9.3791,  -9.9774,  -6.0243],
         [-10.7885, -10.6272, -10.9118,  ...,  -8.5793, -10.5830,  -5.4193],
         [-10.3052, -11.3263, -10.4285,  ...,  -9.6418,  -9.9589,  -7.2134]]],
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)


In [3]:
import json

with open('data/dataset.jsonl', 'r', encoding='utf-8') as f:
    data = [json.loads(line) for line in f]

In [4]:
from utils import Score
scorer = Score(bert_model=model, bert_tokenizer=tokenizer)

In [22]:
# Sens-Making of BERT
correct = 0
total = len(data)

for item in data:
    sentence0 = item['sentence0']
    sentence1 = item['sentence1']
    is_false = item['false']  # 0 means sentence0 is false, 1 means sentence1 is false

    score0 = scorer.bert_score(sentence0)
    score1 = scorer.bert_score(sentence1)

    predicted_true = 0 if score0 < score1 else 1

    if predicted_true != is_false:
        correct += 1

accuracy = correct / total
print(f"BERT Sense-Making Task Accuracy: {accuracy:.4f}")

BERT Sense-Making Task Accuracy: 0.6957


In [5]:
# concatenate
sentences = []
labels = []

for item in data:
        false_sentence = item['sentence0'] if item['false'] == 0 else item['sentence1']
        reasons = [
            f"{false_sentence} is against common sense because {item['A']}.",
            f"{false_sentence} is against common sense because {item['B']}.",
            f"{false_sentence} is against common sense because {item['C']}."
        ]
        
        # map (A, B, C) to index (0, 1, 2)
        label = ord(item['reason']) - ord('A')
        
        sentences.append(reasons)
        labels.append(label)

In [6]:
# Explanation of BERT
correct = 0
total = len(data)

for i, label in enumerate(labels):
    score0 = scorer.bert_score(sentences[i][0])
    score1 = scorer.bert_score(sentences[i][1])
    score2 = scorer.bert_score(sentences[i][2])
    
    if score0 < score1 and score0 < score2:
        predicted_true = 0
    elif score1 < score0 and score1 < score2:
        predicted_true = 1
    else:
        predicted_true = 2
    
    if predicted_true == label:
        correct += 1

accuracy = correct / total
print(f"BERT Explanation Task Accuracy: {accuracy:.4f}")

BERT Explanation Task Accuracy: 0.3741
